In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [13]:
# Look more closely at SPECIAL_CONSIDERATIONS column
application_df["SPECIAL_CONSIDERATIONS"].value_counts()


N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [14]:
# Look more closely at STATUS column
application_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [15]:
# Look more closely at IS_SUCCESSFUL column
application_df["IS_SUCCESSFUL"].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [16]:
# Look more closely at AFFILIATION column
application_df["AFFILIATION"].value_counts()


Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [26]:
# Look more closely at USE_CASE column
use_case_counts = application_df["USE_CASE"].value_counts()
use_case_counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [18]:
# Look more closely at ASK_AMT column
application_df["ASK_AMT"].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

## Preprocessing

In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
# Also drop column 'STATUS' and 'SPECIAL_CONSIDERATIONS' during optimization.
application_df = application_df.drop(["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"], 1)

<ipython-input-19-753cba92828c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"], 1)


In [20]:
# Confirm columns were dropped
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [21]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [22]:
# Look at INCOME_AMT. ## Note to check that the 0 amounts are correct and not missing data.
income_counts = application_df["INCOME_AMT"].value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [23]:
# Prepare Income_Amt for binning counts below 500
income_amt_to_replace = income_counts[income_counts<500].index
income_amt_to_replace

Index(['10M-50M', '5M-10M', '50M+'], dtype='object')

In [24]:
# Replace those income amounts in dataframe with "5M+"
for i in income_amt_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(i,"5M+")

# Confirm binning was successful - Other replaced application types with fewer than 500 entries
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [28]:
# Prepare Use_Case for binning counts below 500
use_case_to_replace = use_case_counts[use_case_counts<500].index
use_case_to_replace

Index(['CommunityServ', 'Heathcare', 'Other'], dtype='object')

In [30]:
# Replace those use cases in dataframe with "Other"
for i in use_case_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(i,"Other")

# Confirm binning was successful - Other replaced application types with fewer than 500 entries
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [31]:
# Look at APPLICATION_TYPE value counts for binning
appl_counts = application_df["APPLICATION_TYPE"].value_counts()
appl_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Choosing a cutoff value of 600, create a list of application types to be replaced by "Other" - Increased to 600 during optimization
application_types_to_replace = appl_counts[appl_counts<600].index
application_types_to_replace

Index(['T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [33]:
# Replace those application types in dataframe with "Other"
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Confirm binning was successful - Other replaced application types with fewer than 500 entries
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [35]:
# Zoom in on smallest CLASSIFICATION value counts
class_counts[class_counts<500]

C1700    287
C4000    194
C5000    116
C1270    114
C2700    104
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 65, dtype: int64

In [36]:
# Choose a cutoff value and create a list of classifications to be replaced. Cutoff increased to 500 during optimization.
classifications_to_replace = class_counts[class_counts<500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Confirm binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [37]:
# Review the types of data in each column
appl_dtypes = application_df.dtypes
appl_dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [38]:
# Create list of columns with object data types
categ_cols = appl_dtypes[appl_dtypes == "object"].index.to_list()
categ_cols

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [39]:
# Convert categorical data to numeric with `pd.get_dummies`
new_appl_df = pd.get_dummies(application_df,columns=categ_cols)
new_appl_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34295,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34296,5000,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34297,5000,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [40]:
# Split our preprocessed data into our features (X) and target (y) arrays
X = new_appl_df.drop(columns="IS_SUCCESSFUL").values
y = new_appl_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=300, activation="sigmoid", input_dim=len(X_train[0])))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=150, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               11100     
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 1)                 151       
                                                                 
Total params: 56,401
Trainable params: 56,401
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5913 - accuracy: 0.7082
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5676 - accuracy: 0.7247
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5638 - accuracy: 0.7262
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5603 - accuracy: 0.7278
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5587 - accuracy: 0.7276
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5567 - accuracy: 0.7290
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5554 - accuracy: 0.7301
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5547 - accuracy: 0.7287
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7291
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5524 - accura

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5575 - accuracy: 0.7326 - 978ms/epoch - 4ms/step
Loss: 0.5574585199356079, Accuracy: 0.7325947284698486


In [53]:
# ANOTHER TRY AT OPTIMIZATION

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid", input_dim=len(X_train[0])))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=20, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 100)               3700      
                                                                 
 dense_20 (Dense)            (None, 80)                8080      
                                                                 
 dense_21 (Dense)            (None, 20)                1620      
                                                                 
 dense_22 (Dense)            (None, 1)                 21        
                                                                 
Total params: 13,421
Trainable params: 13,421
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5831 - accuracy: 0.7124
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5610

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5595 - accuracy: 0.7331 - 529ms/epoch - 2ms/step
Loss: 0.5595018863677979, Accuracy: 0.7330612540245056


In [55]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimized.h5')